<a href="https://colab.research.google.com/github/a-forty-two/GAIP_22_June/blob/main/02_cv_azure_computervision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ACCESS STORAGE ACCOUNT using following key
# DefaultEndpointsProtocol=https;AccountName=madhuratcatcherprivate;AccountKey=fyYDZMmCyi2ciaBafoZ3Gg6d8y5tfBK/BMKQyp17dj7yRR2zf3sP44QVAnheq1KBY3usR2Xpz0mu+ASt1CaH2Q==;EndpointSuffix=core.windows.net

In [1]:
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 178 kB 40.6 MB/s 
     |████████████████████████████████| 41 kB 596 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

In [3]:
subscription_key = "d6d6bae3b5e74a13ab84025266c7d66b"
endpoint = "https://hungryhippo.cognitiveservices.azure.com/"

In [4]:
#authenticate our notebook for azure computer vision
cvcredentials = CognitiveServicesCredentials(subscription_key)
computervision_client = ComputerVisionClient(endpoint, cvcredentials)

In [5]:
baseurl = 'https://madhuratcatcherprivate.blob.core.windows.net/lake/'
imgname = ['BSG', 'rickandmorty','logos','robodoggo', 'batman']
imgurl = {}
for i in range(5):
  fname =  baseurl + '0' + str(i+1) + '.jpeg'
  imgurl[imgname[i]] = fname
imgurl

{'BSG': 'https://madhuratcatcherprivate.blob.core.windows.net/lake/01.jpeg',
 'batman': 'https://madhuratcatcherprivate.blob.core.windows.net/lake/05.jpeg',
 'logos': 'https://madhuratcatcherprivate.blob.core.windows.net/lake/03.jpeg',
 'rickandmorty': 'https://madhuratcatcherprivate.blob.core.windows.net/lake/02.jpeg',
 'robodoggo': 'https://madhuratcatcherprivate.blob.core.windows.net/lake/04.jpeg'}

In [6]:
dir(computervision_client)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_client',
 '_deserialize',
 '_serialize',
 'analyze_image',
 'analyze_image_by_domain',
 'analyze_image_by_domain_in_stream',
 'analyze_image_in_stream',
 'api_version',
 'close',
 'config',
 'describe_image',
 'describe_image_in_stream',
 'detect_objects',
 'detect_objects_in_stream',
 'generate_thumbnail',
 'generate_thumbnail_in_stream',
 'get_area_of_interest',
 'get_area_of_interest_in_stream',
 'get_read_result',
 'list_models',
 'read',
 'read_in_stream',
 'recognize_printed_text',
 'recognize_printed_text_in_stream',
 'tag_image',
 'tag_image_in_stream']

In [7]:
computervision_client.__dict__

{'_client': <msrest.service_client.ServiceClient at 0x7f8e10c53250>,
 '_deserialize': <msrest.serialization.Deserializer at 0x7f8e10c68590>,
 '_serialize': <msrest.serialization.Serializer at 0x7f8e10c681d0>,
 'api_version': '3.2',
 'config': <azure.cognitiveservices.vision.computervision._configuration.ComputerVisionClientConfiguration at 0x7f8e10c53a90>}

In [8]:
for imname, imgur in imgurl.items():
  description_results = computervision_client.describe_image(imgur)
  print('Image = ' + imname)
  print()
  # Get the captions (descriptions) from the response, with confidence level
  print("Description of remote image: ")
  if (len(description_results.captions) == 0):
      print("No description detected.")
      print('****')
  else:
      for caption in description_results.captions:
          print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
      print('****')

Image = BSG

Description of remote image: 
'a person standing on a stage with a crowd watching' with confidence 42.13%
****
Image = rickandmorty

Description of remote image: 
'a person and a boy' with confidence 40.46%
****
Image = logos

Description of remote image: 
'logo, company name' with confidence 99.79%
****
Image = robodoggo

Description of remote image: 
'a yellow and black bicycle' with confidence 36.50%
****
Image = batman

Description of remote image: 
'a collage of a man and a cat' with confidence 58.26%
****


In [10]:
for imname, imgur in imgurl.items():
  print(imgur)

https://madhuratcatcherprivate.blob.core.windows.net/lake/01.jpeg
https://madhuratcatcherprivate.blob.core.windows.net/lake/02.jpeg
https://madhuratcatcherprivate.blob.core.windows.net/lake/03.jpeg
https://madhuratcatcherprivate.blob.core.windows.net/lake/04.jpeg
https://madhuratcatcherprivate.blob.core.windows.net/lake/05.jpeg


In [11]:
for imname, imgur in imgurl.items():

  remote_image_features = ["categories"]
  # Call API with URL and features
  try:
    categorize_results_remote = computervision_client.analyze_image(imgur , remote_image_features)
    print(imname)
    # Print results with confidence score
    print("Categories from remote image: ")
    if (len(categorize_results_remote.categories) == 0):
        print("No categories detected.")
        print('*****')
    else:
        for category in categorize_results_remote.categories:
            print("'{}' with confidence {:.2f}%".format(category.name, category.score * 100))
        print('*****')
  except:
    print('Some random error happened. Move on.')
    continue

BSG
Categories from remote image: 
'people_many' with confidence 37.50%
'people_show' with confidence 31.25%
*****
Some random error happened. Move on.
logos
Categories from remote image: 
'others_' with confidence 1.17%
'text_sign' with confidence 74.61%
*****
robodoggo
Categories from remote image: 
'trans_bicycle' with confidence 99.22%
*****
Some random error happened. Move on.


In [12]:
for imname, imgur in imgurl.items():

  # Call API with remote image
  tags_result_remote = computervision_client.tag_image(imgur)
  print(imname)
  # Print results with confidence score
  print("Tags in the remote image: ")
  if (len(tags_result_remote.tags) == 0):
      print("No tags detected.")
      print('****')
  else:
      for tag in tags_result_remote.tags:
          print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))
      print('****')

BSG
Tags in the remote image: 
'clothing' with confidence 97.26%
'person' with confidence 96.12%
'woman' with confidence 92.54%
'man' with confidence 89.96%
'people' with confidence 68.62%
****
rickandmorty
Tags in the remote image: 
'animated cartoon' with confidence 93.96%
'illustration' with confidence 93.01%
'animation' with confidence 92.51%
'cartoon' with confidence 90.05%
'drawing' with confidence 87.26%
'clipart' with confidence 57.45%
'anime' with confidence 40.79%
****
logos
Tags in the remote image: 
'text' with confidence 99.82%
'screenshot' with confidence 96.53%
'font' with confidence 92.99%
'logo' with confidence 92.14%
'graphics' with confidence 86.75%
'brand' with confidence 85.73%
'number' with confidence 85.07%
'symbol' with confidence 84.05%
'design' with confidence 43.87%
****
robodoggo
Tags in the remote image: 
'outdoor' with confidence 97.68%
'ground' with confidence 97.61%
'tree' with confidence 93.16%
'yellow' with confidence 89.25%
'grass' with confidence 70.

In [13]:
for imname, imgur in imgurl.items():
  detect_objects_results_remote = computervision_client.detect_objects(imgur)
  print('*****')
  print(imname)
  # Print detected objects results with bounding boxes
  print("Detecting objects in remote image:")
  if len(detect_objects_results_remote.objects) == 0:
      print("No objects detected.")
  else:
      for object in detect_objects_results_remote.objects:
          print("object at location {}, {}, {}, {}".format( \
          object.rectangle.x, object.rectangle.x + object.rectangle.w, \
          object.rectangle.y, object.rectangle.y + object.rectangle.h))

*****
BSG
Detecting objects in remote image:
object at location 617, 703, 164, 297
object at location 757, 831, 172, 289
object at location 668, 758, 137, 289
object at location 770, 875, 134, 292
object at location 833, 941, 130, 309
*****
rickandmorty
Detecting objects in remote image:
object at location 257, 656, 368, 1037
*****
logos
Detecting objects in remote image:
No objects detected.
*****
robodoggo
Detecting objects in remote image:
object at location 300, 942, 24, 528
*****
batman
Detecting objects in remote image:
object at location 2, 166, 70, 281
object at location 11, 334, 17, 320
object at location 11, 313, 329, 636
object at location 302, 635, 331, 639


In [14]:

for imgname, imgur in imgurl.items():
    
  remote_image_url_faces = imgur
  # Select the visual feature(s) you want.
  remote_image_features = ["faces"]
  # Call the API with remote URL and features
  detect_faces_results_remote = computervision_client.analyze_image(remote_image_url_faces, remote_image_features)
  print('****')
  print(imgname)
  # Print the results with gender, age, and bounding box
  print("Faces in the remote image: ")
  if (len(detect_faces_results_remote.faces) == 0):
      print("No faces detected.")
  else:
      for face in detect_faces_results_remote.faces:
          print("'{}' of age {} at location {}, {}, {}, {}".format(face.gender, face.age, \
          face.face_rectangle.left, face.face_rectangle.top, \
          face.face_rectangle.left + face.face_rectangle.width, \
          face.face_rectangle.top + face.face_rectangle.height))

****
BSG
Faces in the remote image: 
No faces detected.


ComputerVisionErrorResponseException: ignored